In [2]:
import pickle
import warnings
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from utils import *
from gc import collect


warnings.filterwarnings("ignore", message="X does not have valid feature names, but IncrementalPCA was fitted with feature names")
pd.options.mode.chained_assignment = None

In [33]:
full = pd.read_csv('data/test_set_VU_DM.csv', parse_dates=['date_time'])
enc = np.load('data/enc_test.npy')

full = full.drop(columns=['site_id', 'visitor_location_country_id', 'prop_country_id'])
full = split_time(fillnans(full))
full.head()

,srch_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,...,comp3_rate_missing,comp4_rate_missing,comp5_rate_missing,comp6_rate_missing,comp7_rate_missing,comp8_rate_missing,year,month,day,time
0,1,-1.0,-1.0,3180,3,4.5,1,2.94,0.0691,5.03,...,1,1,1,1,1,1,13,2,2,0.644213
1,1,-1.0,-1.0,5543,3,4.5,1,2.64,0.0843,4.93,...,1,1,1,1,1,1,13,2,2,0.644213
2,1,-1.0,-1.0,14142,2,3.5,1,2.71,0.0556,4.16,...,1,1,0,1,1,1,13,2,2,0.644213
3,1,-1.0,-1.0,22393,3,4.5,1,2.40,0.0561,5.03,...,1,1,0,1,1,1,13,2,2,0.644213
4,1,-1.0,-1.0,24194,3,4.5,1,2.94,0.2090,4.72,...,1,1,0,1,1,1,13,2,2,0.644213


In [40]:
r = torch.load('models/ranker.pt')
data = DataLoader(df, 2**17)

In [99]:
Y = list()
for x in tqdm(data):
    t = r(pipeline(full.loc[data[0].values.flatten()], enc, ret='numpy').reshape(data.bs, 2, -1))
    x['pred'] = t.detach().cpu().numpy().squeeze().round().astype(int)
    x['id'] = x.apply(lambda x: x.iloc[x.pred], axis=1)
    x[['srch', 'prop']] = full.loc[x.id][['srch_id', 'prop_id']].to_numpy()
    y.append(x)

In [99]:
df = pd.concat(Y)
df.groupby(['srch', 'prop']).count()

,first,second,pred,id,srch,prop
0,1,0,1,0,1,3180
1,2,0,1,0,1,3180
2,3,0,1,0,1,3180
3,4,0,1,0,1,3180
4,5,0,1,0,1,3180
...,...,...,...,...,...,...
3,9495,9491,1,9491,620,37240
4,9496,9491,1,9491,620,37240
5,9497,9491,1,9491,620,37240
6,9498,9491,1,9491,620,37240


In [ ]:
net = torch.load('MLP.pt').to('cuda')

In [22]:
Y = list()
for i in tqdm(range(0, len(full), 2**17)):
    x, y = pipeline(full.iloc[i:i+2**17], ret='pandas')
    y['pred'] = 5 * nn.functional.sigmoid(net(torch.tensor(x.to_numpy(), dtype=torch.float32, device='cuda'))).detach().cpu()
    Y.append(y)

100%|██████████████████████████████████████████████████████████████████████████████████| 38/38 [02:04<00:00,  3.27s/it]


In [41]:
df = pd.concat(Y).groupby('srch_id').apply(lambda x: x.sort_values('pred'), include_groups=False)

In [45]:
df.reset_index(0)[['srch_id', 'prop_id']].to_csv('ans.csv', index=False)